In [1]:
import tensorflow as tf
from tensorflow.contrib.layers import batch_norm, fully_connected
from tensorflow.contrib.framework import arg_scope
import time

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

is_training = tf.placeholder(tf.bool, shape=(), name='is_training')
bn_params = {
    'is_training': is_training,
    'decay': 0.99,
    'updates_collections': None
}

with arg_scope([fully_connected], normalizer_fn=batch_norm, normalizer_params=bn_params):
    hidden1 = fully_connected(X, n_hidden1, scope='hidden1')
    hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2')
    logits = fully_connected(hidden2, n_outputs, scope='logits', activation_fn=None)

Defining a loss function

In [2]:
with tf.name_scope("loss"):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(cross_entropy, name="loss")

Defining an optimizer

In [3]:
learning_rate = 0.01

with tf.name_scope("train"):
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

Model evaluation

In [4]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

Initializer and saver

In [5]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Get the MNIST data

In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
n_epochs = 60
batch_size = 50

with tf.Session() as sess:
    time_start = time.time()
    init.run()
    for epoch in range(n_epochs):
        for interation in range(mnist.train.num_examples // batch_size):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: x_batch, y: y_batch, is_training: True})
        acc_train = accuracy.eval(feed_dict={X: x_batch, y: y_batch, is_training: False})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                          y: mnist.validation.labels, is_training: False})
        print(f"{epoch} ({time.time() - time_start}): train accuracy: {acc_train}, Val accuracy {acc_val}")
        
    save_path = saver.save(sess, "./handson-ml-10-mnist-checkpoint")

0 (6.621690034866333): train accuracy: 0.9800000190734863, Val accuracy 0.977400004863739
1 (13.05224347114563): train accuracy: 1.0, Val accuracy 0.9815999865531921
2 (19.480888605117798): train accuracy: 0.9800000190734863, Val accuracy 0.9825999736785889
3 (25.93266534805298): train accuracy: 0.9599999785423279, Val accuracy 0.9836000204086304
4 (32.387468338012695): train accuracy: 0.9800000190734863, Val accuracy 0.9843999743461609
5 (38.855138063430786): train accuracy: 1.0, Val accuracy 0.9833999872207642
6 (45.30400896072388): train accuracy: 0.9800000190734863, Val accuracy 0.9837999939918518
7 (51.73548245429993): train accuracy: 1.0, Val accuracy 0.9851999878883362
8 (58.16939663887024): train accuracy: 1.0, Val accuracy 0.98580002784729
9 (64.61500477790833): train accuracy: 1.0, Val accuracy 0.9851999878883362
10 (71.07843708992004): train accuracy: 1.0, Val accuracy 0.9876000285148621
11 (77.5078489780426): train accuracy: 1.0, Val accuracy 0.9855999946594238
12 (83.96090